In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import itertools
from pmdarima import auto_arima

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Plot_Box_sns
import GrubbsTest
import DickeyFullerTest as dft

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv(os.path.join(Utilities.get_local_data_dir(), r'my_usage\my_usage_2018_h1.csv'))

In [ ]:
df.shape

In [ ]:
df['aep_derived_uom'].value_counts()

In [ ]:
df['aep_srvc_qlty_idntfr'].value_counts()

In [ ]:
df['starttimeperiod'] = pd.to_datetime(df['starttimeperiod'])

In [ ]:
df.columns

In [ ]:
df_vlt = df[df['aep_derived_uom']=='VOLT']
df_kwh = df[df['aep_derived_uom']=='KWH']

In [ ]:
df_vlt['aep_srvc_qlty_idntfr'].value_counts()

In [ ]:
df_kwh['aep_srvc_qlty_idntfr'].value_counts()

In [ ]:
print(f'Before combine_kwh_delivered_and_received_values: {df_kwh.shape}')
df_kwh = AMINonVee.combine_kwh_delivered_and_received_values(df_kwh)
print(f'After combine_kwh_delivered_and_received_values:  {df_kwh.shape}')

In [ ]:
df

In [ ]:
print_re_demo = True
if print_re_demo:
    import reExamples
    reExamples.print_sub_demo()

In [ ]:
rename_files = False
base_dir_w_files_to_rename = os.path.join(Utilities.get_local_data_dir(), r'sample_outages_full\outg_rec_nb_11770899')
find_file_pattern = r'my_usage_*.csv'
repl_pattern = r'(.*)my_usage([^.csv]*)' #This ensures only last occurrence of 'my_usage' before .csv is replaced
repl = r'\1outg_rec_nb_11770899\2'
if rename_files:
    file_paths = Utilities.find_all_paths(base_dir=base_dir_w_files_to_rename, glob_pattern=find_file_pattern)
    for file_path in file_paths:
        new_path = re.sub(repl_pattern, repl, file_path)
        print(f'Renaming: {file_path}')
        print(f'To:       {new_path}\n')
        os.rename(file_path, new_path)

In [ ]:
my_usage_dir = os.path.join(Utilities.get_local_data_dir(), r'my_usage')

In [ ]:
my_usages_csvs = Utilities.find_all_paths(base_dir=my_usage_dir, glob_pattern=r'my_usage_*.csv')

In [ ]:
len(my_usages_csvs)

In [ ]:
dfs_vlt = []
dfs_kwh = []
for csv in my_usages_csvs:
    df = pd.read_csv(csv)
    if df.shape[0]==0:
        continue
    df['starttimeperiod'] = pd.to_datetime(df['starttimeperiod'])
    df_vlt = df[df['aep_derived_uom']=='VOLT']
    df_kwh = df[df['aep_derived_uom']=='KWH']
    df_kwh = AMINonVee.combine_kwh_delivered_and_received_values(df_kwh)
    
    dfs_vlt.append(df_vlt)
    dfs_kwh.append(df_kwh)

In [ ]:
df_vlt = pd.concat(dfs_vlt)
df_kwh = pd.concat(dfs_kwh)

In [ ]:
df_vlt

In [ ]:
df_vlt['starttimeperiod_tz_aware'] = pd.to_datetime(df_vlt['starttimeperiod'], utc=True)
df_vlt['starttimeperiod_local'] = df_vlt['starttimeperiod_tz_aware'].dt.tz_localize(None)
df_vlt['starttimeperiod_utc'] = df_vlt['starttimeperiod_tz_aware'].dt.tz_convert(None)
df_vlt = df_vlt.set_index('starttimeperiod_local', drop=False).sort_index()

In [ ]:
df_kwh['starttimeperiod_tz_aware'] = pd.to_datetime(df_kwh['starttimeperiod'], utc=True)
df_kwh['starttimeperiod_local'] = df_kwh['starttimeperiod_tz_aware'].dt.tz_localize(None)
df_kwh['starttimeperiod_utc'] = df_kwh['starttimeperiod_tz_aware'].dt.tz_convert(None)
df_kwh = df_kwh.set_index('starttimeperiod_local', drop=False).sort_index()

In [ ]:
ax = df_vlt['2018-06-26T00:00:00':'2018-06-27T00:00:00']['value'].plot()
ax.tick_params(axis='x', labelrotation=45);

In [ ]:
ax = df_kwh['2018-06-25T00:00:00':'2018-06-28T00:00:00']['value'].plot()
ax.tick_params(axis='x', labelrotation=45);

In [ ]:
tmp_df = df_kwh.resample('H').mean()

In [ ]:
ax = tmp_df['2018-06-25T00:00:00':'2018-06-28T00:00:00']['value'].plot()
ax.tick_params(axis='x', labelrotation=45);

In [ ]:
ax = tmp_df['2018-06-23T00:00:00':'2018-06-30T00:00:00']['value'].plot()
ax.tick_params(axis='x', labelrotation=45);

In [ ]:
ax = tmp_df['2018-06-15T00:00:00':'2018-07-15T00:00:00']['value'].plot()
ax.tick_params(axis='x', labelrotation=45);

In [ ]:
ax = df_kwh.resample('M').mean()['value'].plot()
ax.tick_params(axis='x', labelrotation=45);

In [ ]:
ax = df_kwh.resample('W-Mon').mean()['value'].plot()
ax.tick_params(axis='x', labelrotation=45);

In [ ]:
ax = df_kwh.resample('2W-Mon').mean()['value'].plot()
ax.tick_params(axis='x', labelrotation=45);

In [ ]:
ax = df_kwh['2019-07-25T00:00:00':'2019-07-27T00:00:00']['value'].plot()
ax.tick_params(axis='x', labelrotation=45);

In [ ]:
ax = df_kwh['2020-07-25T00:00:00':'2020-07-27T00:00:00']['value'].plot()
ax.tick_params(axis='x', labelrotation=45);

In [ ]:
ax = df_vlt['2018-06-24T00:00:00':'2018-06-29T00:00:00']['value'].plot()
ax.tick_params(axis='x', labelrotation=45);

In [ ]:
ax = df_kwh['2018-06-24T00:00:00':'2018-06-29T00:00:00']['value'].plot()
ax.tick_params(axis='x', labelrotation=45);

In [ ]:
ax = tmp_df['2018-07-15T00:00:00':'2018-07-29T00:00:00']['value'].plot()
ax.tick_params(axis='x', labelrotation=45);

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import acovf, acf, pacf, pacf_yw, pacf_ols
from pandas.plotting import lag_plot

In [ ]:
df_kwh_sub = df_kwh['2018-07-01T00:00:00':'2018-07-07T00:00:00'].copy()

In [ ]:
df_kwh_sub['value']

In [ ]:
lag_plot(df_kwh_sub['value'])

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
plot_acf(df_kwh_sub['value'], lags=40);

In [ ]:
plot_acf(df_kwh_sub['value'], lags=200);

In [ ]:
plot_pacf(df_kwh_sub['value'], lags=40);

In [ ]:
df_kwh_sub['value'].plot()

In [ ]:
dft.adf_test(df_kwh_sub['value'])
df_kwh_sub['value'].plot()

In [ ]:
dft.adf_test(df_kwh.resample('M').mean()['value'])
df_kwh.resample('M').mean()['value'].plot()

In [ ]:
from statsmodels.graphics.tsaplots import month_plot, quarter_plot, seasonal_plot

In [ ]:
df_kwh_sub_gpd = df_kwh_sub.groupby(pd.Grouper(freq='H'))

In [ ]:
# seasonal_plot(df_kwh_sub_gpd, df_kwh_sub_gpd.groups.keys())

In [ ]:
month_plot(df_kwh.resample('M').mean()['value']);

In [ ]:
df_kwh.resample('M').mean()['value'].plot()

In [ ]:
from pmdarima import auto_arima
from statsmodels.tsa.arima_model import ARMA, ARIMA, ARMAResults, ARIMAResults
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
#tmp_df = df_kwh_sub.resample('H').mean()
tmp_df = df_kwh['2018-07-01T00:00:00':'2018-07-28T00:00:00'].resample('H').mean()
tmp_df = tmp_df[tmp_df['value'].notna()]

In [ ]:
#auto_arima(df_kwh_sub['value']).summary()
#auto_arima(df_kwh_sub['value'], seasonal=True, m=96).summary()
auto_arima(tmp_df['value'], seasonal=True, m=24).summary()

In [ ]:
tmp_df['value'].plot()

In [ ]:
train = tmp_df.iloc[:round(0.8*len(tmp_df))]
test = tmp_df.iloc[round(0.8*len(tmp_df)):]

In [ ]:
model = SARIMAX(train['value'], order=(1,0,0), seasonal_order=(2, 0, 1, 24))

In [ ]:
results = model.fit()

In [ ]:
results.summary()

In [ ]:
predictions = results.predict(len(train), len(tmp_df), typ='levels')

In [ ]:
tmp_df['value'].plot(figsize=(12,6))
predictions.plot()

In [ ]:
tmp_df.iloc[len(train):len(tmp_df)]['value'].plot(figsize=(12,6))
predictions.plot()

In [ ]:
all_predictions = results.predict(0, len(tmp_df), typ='levels')
tmp_df['value'].plot(figsize=(12,6))
all_predictions.plot()

In [ ]:
tmp_df

In [ ]:
# predictions2 = results.predict(0, 2*len(tmp_df), typ='levels')

In [ ]:
# df_kwh_sub['value'].plot()
# predictions2.plot()

In [ ]:
df_kwh['2018-07-01T00:00:00':'2018-08-01T00:00:00'].resample('D').mean()['value'].plot()

In [ ]:
df_kwh.resample('MS').mean()['value'].plot()

In [ ]:
import keras

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
df_kwh.shape

In [ ]:
df_kwh = df_kwh[df_kwh['value'].notna()]
df_kwh.shape

In [ ]:
df_kwh_D = df_kwh.resample('D').mean()
print(df_kwh_D.shape)

df_kwh_D = df_kwh_D[df_kwh_D['value'].notna()]
print(df_kwh_D.shape)

In [ ]:
df_kwh_H = df_kwh.resample('H').mean()
print(df_kwh_H.shape)

df_kwh_H = df_kwh_H[df_kwh_H['value'].notna()]
print(df_kwh_H.shape)

In [ ]:
# train = df_kwh.iloc[:round(0.8*len(df_kwh))][['value']]
# test = df_kwh.iloc[round(0.8*len(df_kwh)):][['value']]

In [ ]:
# train = df_kwh_D.iloc[:round(0.8*len(df_kwh_D))][['value']]
# test = df_kwh_D.iloc[round(0.8*len(df_kwh_D)):][['value']]

In [ ]:
# train = df_kwh_H.iloc[:round(0.8*len(df_kwh_H))][['value']]
# test = df_kwh_H.iloc[round(0.8*len(df_kwh_H)):][['value']]

train = df_kwh_H.iloc[:2000][['value']]
test = df_kwh_H.iloc[2000:2500][['value']]

In [ ]:
type(train)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(train)

scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

In [ ]:
plt.plot(range(len(scaled_train)), scaled_train)

In [ ]:
df_kwh.iloc[:48]['value'].plot()

In [ ]:
#scaled_train

In [ ]:
# Let's define to get 14 days back and then predict the next day out
n_input = 24
n_features = 1
generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)

In [ ]:
# # define model
# model = Sequential()
# model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
# model.add(Dense(1))
# model.compile(optimizer='adam', loss='mse')

In [ ]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features), return_sequences=True))
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features), return_sequences=True))
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

In [ ]:
# fit model
model.fit_generator(generator,epochs=5)

In [ ]:
loss_per_epoch = model.history.history['loss']
plt.plot(range(len(loss_per_epoch)),loss_per_epoch)

In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

for i in range(len(test)):
    
    # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    current_pred = model.predict(current_batch)[0]
    
    # store prediction
    test_predictions.append(current_pred) 
    
    # update batch to now include prediction and drop first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
true_predictions = scaler.inverse_transform(test_predictions)

In [ ]:
# IGNORE WARNINGS
test['Predictions'] = true_predictions

In [ ]:
test.plot(figsize=(12,8))

In [ ]:
all_predictions=[]

first_eval_batch = scaled_train[:n_input]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

for i in range(n_input,len(scaled_train)):
    
    # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    current_pred = model.predict(current_batch)[0]
    
    # store prediction
    all_predictions.append(current_pred) 
    
    # update batch to now include prediction and drop first value
    #current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)
    current_batch = np.append(current_batch[:,1:,:],[[scaled_train[i]]],axis=1)

In [ ]:
all_true_predictions = scaler.inverse_transform(all_predictions)

In [ ]:
all_true_predictions_df = pd.DataFrame(data=all_true_predictions, index=train.index[n_input:])

In [ ]:
ax=train.plot(figsize=(12,6))
all_true_predictions_df.plot(ax=ax)

In [ ]:
plot_pacf(df_kwh_H['value'], lags=40, zero=False);

In [ ]:
plot_acf(df_kwh_H['value'], lags=40);